In [1]:
from fpl import Loader, Player, Team, ExpectedPointsCalculator, Optimizer
import heapq

### Dowloading your personal FPL team

You will need your login details and manager id. To get your manager id:
1. Log on to [Fantasy Premier League](https://fantasy.premierleague.com/) and navigate to the points tab.
2. Your `manager_id` should appear somewhere in the URL e.g. 3247546 . [FPL APIs Explained](https://www.oliverlooney.com/blogs/FPL-APIs-Explained#how-to-use-authenticated-endpoints) has some screenshots of this.


In [4]:
team = Loader.get_my_team_from_api(login="email", password="password", manager_id=123456)
print(team)

---------------------------------------------------------------------------
{GKPS}
Player(element=201, name='Henderson', position=1, club=7, cost=44)
Player(element=521, name='Fabianski', position=1, club=19, cost=41)
---------------------------------------------------------------------------
{DEFS}
Player(element=70, name='Kerkez', position=2, club=3, cost=47)
Player(element=231, name='Mykolenko', position=2, club=8, cost=43)
Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72)
Player(element=399, name='Hall', position=2, club=15, cost=47)
Player(element=475, name='Taylor', position=2, club=17, cost=39)
---------------------------------------------------------------------------
{MIDS}
Player(element=99, name='Mbeumo', position=3, club=4, cost=78)
Player(element=182, name='Palmer', position=3, club=6, cost=109)
Player(element=328, name='M.Salah', position=3, club=12, cost=132)
Player(element=366, name='B.Fernandes', position=3, club=14, cost=84)
Player(element=585

If the above API request fails, you'll need a more manual solution:
1. Log on to [Fantasy Premier League](https://fantasy.premierleague.com/)
2. View the local json by typing this URL `https://fantasy.premierleague.com/api/my-team/{manager_id}/` into your browser with the appropriate `manager_id`
3. Save the json locally for example as `my_team.json`

In [4]:
team = Loader.get_my_team_from_local(local_filename="resources/my_team.json")
print(team)

---------------------------------------------------------------------------
{GKPS}
Player(element=201, name='Henderson', position=1, club=7, cost=44)
Player(element=521, name='Fabianski', position=1, club=19, cost=41)
---------------------------------------------------------------------------
{DEFS}
Player(element=70, name='Kerkez', position=2, club=3, cost=47)
Player(element=231, name='Mykolenko', position=2, club=8, cost=43)
Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72)
Player(element=399, name='Hall', position=2, club=15, cost=47)
Player(element=475, name='Taylor', position=2, club=17, cost=39)
---------------------------------------------------------------------------
{MIDS}
Player(element=99, name='Mbeumo', position=3, club=4, cost=78)
Player(element=182, name='Palmer', position=3, club=6, cost=109)
Player(element=328, name='M.Salah', position=3, club=12, cost=132)
Player(element=366, name='B.Fernandes', position=3, club=14, cost=84)
Player(element=585

### Candidate list

Probably good to wrap up a way of converting an id into a player.

In [3]:
Loader.find_matching_players("gordon")

[(Player(element=398, name='Gordon', position=3, club=15, cost=77),
  'Anthony Gordon')]

In [4]:
Player(element=398, name='Gordon', position=3, club=15, cost=77)

Player(element=398, name='Gordon', position=3, club=15, cost=77)

In [5]:
candidates_gkps = [235] + [201, 521]
candidates_defs = [] + [70, 231, 311, 399, 475]
candidates_mids = [364, 398] + [99, 182, 328, 366, 585]
candidates_fwds = [321, 207, 220] + [129, 401, 447]

### Expected Points Calculator

This is a concrete implementation of an expected points calculator. It should probably be wrapped up and tested.

Perhaps the expected points calculator should be able to take a player as an input.

In [6]:
class SimpleExpectedPointsCalculator(ExpectedPointsCalculator):
    """Simple expected points calculator.
    Major assumption is that the candidates you put in this list are nailed on to play 90.
    """
    fixture_difficulty_to_adj_map = {
        2: 1.5,
        3: 0.75,
        4: -0.75,
        5: -1.5
    }
    
    @staticmethod
    def get_expected_points(player_id: int, gameweek: int) -> float:
        basic_info = Loader.get_player_basic_info(player_id)
        chance_of_playing_next_round = basic_info["chance_of_playing_next_round"] / 100.0 if basic_info["chance_of_playing_next_round"] is not None else 1.0
        points_per_game = float(basic_info["points_per_game"])
        form = float(basic_info["form"])
        fixtures = Loader.get_player_future_info_for_gameweek(player_id, gameweek)
        expected_points = 0
        for fixture in fixtures:
            difficulty = fixture["difficulty"]
            difficulty_adj = SimpleExpectedPointsCalculator.fixture_difficulty_to_adj_map[difficulty]
            expected_points += (0.25 * form + 0.75 * points_per_game) + difficulty_adj
            
        return chance_of_playing_next_round * expected_points

### How to calculate points

The below could probably be put into functions somewhere

In [7]:
def get_discounted_reward(
    player_id: int, number_gameweeks: int, gamma: float = 1
) -> float:
    """Gets the discounted reward of a candidate over a certain number of gameweeks
    :param player_id: the player which you want the reward for
    :param number_gameweeks: how many gameweeks do you want to consider
    :param gamma: discount factor

    :return: discounted reward
    """
    discounted_reward = 0
    discount_factor = 1
    next_gameweek = Loader.get_next_gameweek()
    for gw in range(next_gameweek, next_gameweek + number_gameweeks):
        discounted_reward += (
            discount_factor
            * SimpleExpectedPointsCalculator.get_expected_points(player_id, gw)
        )
        discount_factor *= gamma

    return discounted_reward

def print_sorted_candidates(
    candidates: list[int], number_gameweeks: int, gamma: float = 1
) -> None:
    """Print out the list of players and rewards they get
    :param candidates: list of player ids
    :param number_gameweeks: how many gameweeks do you want to consider
    :param gamma: discount factor
    """
    sorted_candidates = []
    for i in candidates:
        r = get_discounted_reward(i, number_gameweeks, gamma)
        sorted_candidates.append((i, r))

    sorted_candidates = reversed(sorted(sorted_candidates, key=lambda x: x[1]))
    for i, r in sorted_candidates:
        web_name = Loader.get_player_basic_info(i)["web_name"]
        print(f"id: {i}, name: {web_name}, reward: {r}")

    return

In [8]:
print_sorted_candidates(
    candidates_gkps + candidates_defs + candidates_mids + candidates_fwds, 5, 0.80
)

id: 328, name: M.Salah, reward: 41.70952
id: 182, name: Palmer, reward: 26.73724
id: 99, name: Mbeumo, reward: 26.339840000000006
id: 401, name: Isak, reward: 25.94052
id: 311, name: Alexander-Arnold, reward: 23.82696
id: 321, name: Gakpo, reward: 23.390800000000006
id: 447, name: Wood, reward: 21.861120000000003
id: 364, name: Amad, reward: 20.947680000000002
id: 207, name: Mateta, reward: 19.816200000000006
id: 235, name: Pickford, reward: 19.415760000000002
id: 366, name: B.Fernandes, reward: 17.8382
id: 398, name: Gordon, reward: 17.45248
id: 70, name: Kerkez, reward: 16.64648
id: 201, name: Henderson, reward: 15.614200000000004
id: 231, name: Mykolenko, reward: 15.599360000000004
id: 129, name: João Pedro, reward: 15.59648
id: 399, name: Hall, reward: 14.931280000000001
id: 585, name: I.Sarr, reward: 14.68976
id: 521, name: Fabianski, reward: 8.77688
id: 220, name: Calvert-Lewin, reward: 6.654760000000001
id: 475, name: Taylor, reward: 3.1783599999999996


### Optimizing the team

Maybe part of the optimization functionality should check the candidate ids are not in the list.

In [9]:
candidates = []
for pid in candidates_gkps + candidates_defs + candidates_mids + candidates_fwds:
    d = {
        "element": pid,
        "name": Loader.get_player_basic_info(pid)["web_name"],
        "position": Loader.get_player_basic_info(pid)["element_type"],
        "club": Loader.get_player_basic_info(pid)["team"],
        "cost": Loader.get_player_basic_info(pid)["now_cost"],
    }
    player = Player(**d)
    if player.element in [p.element for p in gkps|defs|mids|fwds]:
        print(f"Already have: {player}")
        continue
    else:
        candidates.append(player)
        
print("Candidates:")
candidates

Already have: Player(element=201, name='Henderson', position=1, club=7, cost=45)
Already have: Player(element=521, name='Fabianski', position=1, club=19, cost=41)
Already have: Player(element=70, name='Kerkez', position=2, club=3, cost=49)
Already have: Player(element=231, name='Mykolenko', position=2, club=8, cost=44)
Already have: Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=73)
Already have: Player(element=399, name='Hall', position=2, club=15, cost=51)
Already have: Player(element=475, name='Taylor', position=2, club=17, cost=39)
Already have: Player(element=99, name='Mbeumo', position=3, club=4, cost=78)
Already have: Player(element=182, name='Palmer', position=3, club=6, cost=113)
Already have: Player(element=328, name='M.Salah', position=3, club=12, cost=137)
Already have: Player(element=366, name='B.Fernandes', position=3, club=14, cost=84)
Already have: Player(element=585, name='I.Sarr', position=3, club=7, cost=57)
Already have: Player(element=129, n

[Player(element=235, name='Pickford', position=1, club=8, cost=51),
 Player(element=364, name='Amad', position=3, club=14, cost=56),
 Player(element=398, name='Gordon', position=3, club=15, cost=77),
 Player(element=321, name='Gakpo', position=4, club=12, cost=76),
 Player(element=207, name='Mateta', position=4, club=7, cost=74),
 Player(element=220, name='Calvert-Lewin', position=4, club=8, cost=54)]

In [10]:
top_three = Optimizer.optimize_team(
    team,
    candidates,
    SimpleExpectedPointsCalculator,
    gameweek=Loader.get_next_gameweek(),
    horizon=3,
    max_transfers=2,
    gamma=0.8,
    wildcard=False,
)
third_best_score, third_best_team = heapq.heappop(top_three)
second_best_score, second_best_team = heapq.heappop(top_three)
first_best_score, first_best_team = heapq.heappop(top_three)

In [11]:
print("Score: {}".format(first_best_score))
print(first_best_team)

Score: 224.95300000000003
---------------------------------------------------------------------------
{GKPS}
Player(element=201, name='Henderson', position=1, club=7, cost=44)
Player(element=521, name='Fabianski', position=1, club=19, cost=41)
---------------------------------------------------------------------------
{DEFS}
Player(element=70, name='Kerkez', position=2, club=3, cost=47)
Player(element=231, name='Mykolenko', position=2, club=8, cost=43)
Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72)
Player(element=399, name='Hall', position=2, club=15, cost=47)
Player(element=475, name='Taylor', position=2, club=17, cost=39)
---------------------------------------------------------------------------
{MIDS}
Player(element=99, name='Mbeumo', position=3, club=4, cost=78)
Player(element=182, name='Palmer', position=3, club=6, cost=109)
Player(element=328, name='M.Salah', position=3, club=12, cost=132)
Player(element=364, name='Amad', position=3, club=14, cost=56)

In [12]:
print("Score: {}".format(second_best_score))
print(second_best_team)

Score: 222.33
---------------------------------------------------------------------------
{GKPS}
Player(element=201, name='Henderson', position=1, club=7, cost=44)
Player(element=235, name='Pickford', position=1, club=8, cost=51)
---------------------------------------------------------------------------
{DEFS}
Player(element=70, name='Kerkez', position=2, club=3, cost=47)
Player(element=231, name='Mykolenko', position=2, club=8, cost=43)
Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72)
Player(element=399, name='Hall', position=2, club=15, cost=47)
Player(element=475, name='Taylor', position=2, club=17, cost=39)
---------------------------------------------------------------------------
{MIDS}
Player(element=99, name='Mbeumo', position=3, club=4, cost=78)
Player(element=182, name='Palmer', position=3, club=6, cost=109)
Player(element=328, name='M.Salah', position=3, club=12, cost=132)
Player(element=364, name='Amad', position=3, club=14, cost=56)
Player(elemen

In [13]:
print("Score: {}".format(third_best_score))
print(third_best_team)

Score: 222.01000000000002
---------------------------------------------------------------------------
{GKPS}
Player(element=235, name='Pickford', position=1, club=8, cost=51)
Player(element=521, name='Fabianski', position=1, club=19, cost=41)
---------------------------------------------------------------------------
{DEFS}
Player(element=70, name='Kerkez', position=2, club=3, cost=47)
Player(element=231, name='Mykolenko', position=2, club=8, cost=43)
Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72)
Player(element=399, name='Hall', position=2, club=15, cost=47)
Player(element=475, name='Taylor', position=2, club=17, cost=39)
---------------------------------------------------------------------------
{MIDS}
Player(element=99, name='Mbeumo', position=3, club=4, cost=78)
Player(element=182, name='Palmer', position=3, club=6, cost=109)
Player(element=328, name='M.Salah', position=3, club=12, cost=132)
Player(element=364, name='Amad', position=3, club=14, cost=56)


In [14]:
Optimizer.optimal_formation(first_best_team, SimpleExpectedPointsCalculator, 24)

{'gkps': frozenset({Player(element=201, name='Henderson', position=1, club=7, cost=44)}),
 'defs': frozenset({Player(element=231, name='Mykolenko', position=2, club=8, cost=43),
            Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72),
            Player(element=399, name='Hall', position=2, club=15, cost=47)}),
 'mids': frozenset({Player(element=99, name='Mbeumo', position=3, club=4, cost=78),
            Player(element=182, name='Palmer', position=3, club=6, cost=109),
            Player(element=328, name='M.Salah', position=3, club=12, cost=132),
            Player(element=364, name='Amad', position=3, club=14, cost=56)}),
 'fwds': frozenset({Player(element=321, name='Gakpo', position=4, club=12, cost=76),
            Player(element=401, name='Isak', position=4, club=15, cost=92),
            Player(element=447, name='Wood', position=4, club=16, cost=70)}),
 'captain': Player(element=328, name='M.Salah', position=3, club=12, cost=132),
 'total_exp_point

In [15]:
Optimizer.optimal_formation(second_best_team, SimpleExpectedPointsCalculator, 24)

{'gkps': frozenset({Player(element=235, name='Pickford', position=1, club=8, cost=51)}),
 'defs': frozenset({Player(element=231, name='Mykolenko', position=2, club=8, cost=43),
            Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72),
            Player(element=399, name='Hall', position=2, club=15, cost=47)}),
 'mids': frozenset({Player(element=99, name='Mbeumo', position=3, club=4, cost=78),
            Player(element=182, name='Palmer', position=3, club=6, cost=109),
            Player(element=328, name='M.Salah', position=3, club=12, cost=132),
            Player(element=364, name='Amad', position=3, club=14, cost=56),
            Player(element=585, name='I.Sarr', position=3, club=7, cost=57)}),
 'fwds': frozenset({Player(element=401, name='Isak', position=4, club=15, cost=92),
            Player(element=447, name='Wood', position=4, club=16, cost=70)}),
 'captain': Player(element=328, name='M.Salah', position=3, club=12, cost=132),
 'total_exp_points

In [16]:
Optimizer.optimal_formation(team, SimpleExpectedPointsCalculator, 24)

{'gkps': frozenset({Player(element=201, name='Henderson', position=1, club=7, cost=44)}),
 'defs': frozenset({Player(element=231, name='Mykolenko', position=2, club=8, cost=43),
            Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72),
            Player(element=399, name='Hall', position=2, club=15, cost=47)}),
 'mids': frozenset({Player(element=99, name='Mbeumo', position=3, club=4, cost=78),
            Player(element=182, name='Palmer', position=3, club=6, cost=109),
            Player(element=328, name='M.Salah', position=3, club=12, cost=132),
            Player(element=366, name='B.Fernandes', position=3, club=14, cost=84),
            Player(element=585, name='I.Sarr', position=3, club=7, cost=57)}),
 'fwds': frozenset({Player(element=401, name='Isak', position=4, club=15, cost=92),
            Player(element=447, name='Wood', position=4, club=16, cost=70)}),
 'captain': Player(element=328, name='M.Salah', position=3, club=12, cost=132),
 'total_ex